 # Import libraries

In [ ]:
import pandas as pd
import os
import requests
from io import BytesIO
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ipython-autotime # To measure the execution time of our code
 
%load_ext autotime

time: 160 µs (started: 2021-04-03 07:03:13 +00:00)


# Load the CSV files

In [ ]:
# Read 2019 Jan CSV into a DataFrame
jan2019 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2019_full.csv')

In [ ]:
# Drop the old row indices of the dataframe
jan2019 = jan2019.drop(columns='Unnamed: 0')

# View the dataframe
jan2019

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
0,2018-12-31 23:59:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,48f34eaa179e51989c8f937631da1bce
1,2019-01-01 00:00:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,ed9a57e100a83156605b19ec84bdc2fe
2,2019-01-01 00:01:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c2713943d69482eaa4191454c76af1c8
3,2019-01-01 00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
4,2019-01-01 00:05:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,fa2696a66b3c5bb982e26b765f2d1b98
...,...,...,...,...,...,...,...,...
30229,2019-01-31 23:54:47+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d070313c2ca6bde6c44a074d136dcf05
30230,2019-01-31 23:56:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,41786bfddd62f3138dd9d71b6210af28
30231,2019-01-31 23:57:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,afd6213f51bda947cc27b98082c21f53
30232,2019-01-31 23:58:27+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0c172e9da57d5e8e241d6ca05e25347e


In [ ]:
# Read 2020 Jan CSV into a DataFrame
jan2020 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/future_results_2020_full.csv')

# Drop the old row indices of the dataframe
jan2020 = jan2020.drop(columns='Unnamed: 0')

# View the dataframe
jan2020

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
0,2019-12-31 23:59:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,1cc0aed7f981fccabfe22c43229a48d5
1,2020-01-01 00:00:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9daa712d0a50f989a8a698b6fc7dd33b
2,2020-01-01 00:01:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d623bfdab733b966ade6e390f18cd7fb
3,2020-01-01 00:02:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0ddb5b38083ebcce87c8228b746a7347
4,2020-01-01 00:03:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,fc4923fc71ec9a99940007cea41ecf23
...,...,...,...,...,...,...,...,...
29935,2020-01-31 23:53:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bb50d5020f94a869583863593c0308d8
29936,2020-01-31 23:54:50+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f54b2765d796c244350a4db077e085cf
29937,2020-01-31 23:56:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,24c4ff81022e653f45cfb8bd0af3e276
29938,2020-01-31 23:57:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d85d5d9db869044c3f64b54fa5fcb41d


### Write function getImages


In [ ]:
def getImages(index, row, destination_path): 
    row_num = index # Store the index of the current row in a variable
    temp_url = row['image'] # Store the url of the current image in a variable
    temp_res = requests.get(temp_url) # Make an API request using the image url 

    try:
      temp_img = Image.open(BytesIO(temp_res.content)) # Open the image contained in the url
      temp_img.save(destination_path + '{}.jpg'.format(row_num)) # Save the image to the specified path with the index value as the file name
    except:
      pass

In [ ]:
# Test the getImages function by retrieving the first five images captured in 2019

for index, row in jan2019.iloc[0:5].iterrows():
    getImages(index, row, '/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/car_image_2019_Jan/')

# Full image collection

### Run a concurrent getImages call for 2019

In [ ]:
import concurrent.futures

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=150) as executor:
    future_to_scraper = [executor.submit(getImages, index, row, '/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/car_image_2019_Jan/') for index, row in jan2019.iloc[0:].iterrows()]

### Count the number of images in car_image_2019_Jan

In [ ]:
from os import listdir

FOLDER_PATH = 'car_image_2019_Jan'
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/'

# Check the number of images stored in Google Drive
print(len(os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))))

30233


In [ ]:
# Check the number of rows in the dataframe
len(jan2019)

# The discrepancy between the number of stored images and rows in the dataframe may be simply due to errors in the API

30234

### Repeat the same steps for car_image_2020_Jan



In [ ]:
# Retrieve all images for 2020 Jan

with concurrent.futures.ThreadPoolExecutor(max_workers=150) as executor:
    future_to_scraper = [executor.submit(getImages, index, row, '/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/car_image_2020_Jan/') for index, row in jan2020.iloc[0:].iterrows()]

time: 21min 21s (started: 2021-04-03 07:03:40 +00:00)


In [ ]:
# Count the number of images in car_image_2020_Jan

FOLDER_PATH = 'car_image_2020_Jan'
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic/'
print(len(os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))))

29937
time: 331 ms (started: 2021-04-03 07:35:20 +00:00)


In [ ]:
# Check the number of rows in the dataframe
len(jan2020)

29940

time: 2.53 ms (started: 2021-04-03 07:35:27 +00:00)


# Prepare for OpenCV in GPU mode in Colab
### Run the first cell as specified by [C Kuan](https://towardsdatascience.com/how-to-use-opencv-with-gpu-on-colab-25594379945f)

In [ ]:
%cd /content

!git clone https://github.com/opencv/opencv

!git clone https://github.com/opencv/opencv_contrib

!mkdir /content/build

%cd /content/build

!cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  -DBUILD_SHARED_LIBS=OFF  -DBUILD_TESTS=OFF  -DBUILD_PERF_TESTS=OFF -DBUILD_EXAMPLES=OFF -DWITH_OPENEXR=OFF -DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON /content/opencv

!make -j8 install

/content
Cloning into 'opencv'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (500/500), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 290461 (delta 246), reused 394 (delta 198), pack-reused 289961
Receiving objects: 100% (290461/290461), 478.55 MiB | 33.20 MiB/s, done.
Resolving deltas: 100% (202978/202978), done.
Checking out files: 100% (6790/6790), done.
Cloning into 'opencv_contrib'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 34856 (delta 56), reused 116 (delta 35), pack-reused 34671
Receiving objects: 100% (34856/34856), 130.69 MiB | 36.82 MiB/s, done.
Resolving deltas: 100% (21514/21514), done.
/content/build
-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting 

### Import and check the version of OpenCV

In [ ]:
import cv2
cv2.__version__

'4.5.2-dev'

time: 1.25 s (started: 2021-04-03 10:13:15 +00:00)


### Save the results of installation to your own Google Drive
Perform this step so that you do not have to go through the 2.5-hour installation each time you access Colab.

In [ ]:
# Save the new OpenCV into your own GDrive
# !mkdir  "/content/drive/My Drive/cv2_gpu"
!cp  /content/build/lib/python3/cv2.cpython-37m-x86_64-linux-gnu.so   "/content/drive/My Drive/cv2_gpu" 

time: 3.74 s (started: 2021-04-03 10:20:08 +00:00)


### Copy the OpenCV library into your working directory

In [ ]:
!cp  /content/build/lib/python3/cv2.cpython-37m-x86_64-linux-gnu.so   "/content/drive/MyDrive/Colab Notebooks/UpLevel SG Traffic" 

time: 3.76 s (started: 2021-04-03 10:24:39 +00:00)
